In [ ]:
import typesense
import os
from dotenv import load_dotenv
import yaml
from pathlib import Path

In [ ]:
load_dotenv()

TYPESENSE_KEY = os.environ['typesense_key']

In [ ]:
client = typesense.Client({
  'nodes': [{
    'host': 'localhost',
    'port': '8108',
    'protocol': 'http'
  }],
  'api_key': TYPESENSE_KEY,
  'connection_timeout_seconds': 2
})

In [ ]:
schema_path = Path().cwd().parent / 'ani_dagster/ani_dagster/assets/search_index/index_schema.yml'

In [ ]:
with open(schema_path, 'r') as fp:
    schemas = yaml.safe_load(fp)

In [ ]:
client.collections['entity2'].update({
    'fields': [
    {
      'name'  :  'type',
      'drop'  :  True
    },
    {
      'name'  :  'type',
      'type'  :  'string[]',
      'index' : True,
      'optional': False,
      'facet': True,
    }
    ]
})

In [ ]:
from pydantic import BaseModel
from typing import Any

class Hit(BaseModel):
    document: dict[str, Any]
    text_match: int
    highlights: list[dict[str, Any]]

class TypesenseSearchResult(BaseModel):
    found: int
    hits: list[Hit]
    page: int
    search_time_ms: int

class TypesenseMultiSearchResult(BaseModel):
    results: list[TypesenseSearchResult]


In [ ]:
search_parameters = {
    'q'         : 'naruto',
    'query_by'  : 'title',
#   'filter_by' : 'num_employees:>100',
#   'sort_by'   : 'num_employees:desc'
}

search_hits = client.collections['webpage2'].documents.search(search_parameters)
search_result = TypesenseSearchResult.parse_obj(search_hits)

In [ ]:
search_result

In [ ]:
q = 'naruto movie'
search_requests = {
    'searches': [
    {
        'collection': 'webpage',
        'q': q,
        'query_by': 'title, url, aka',
    },
    {
        'collection': 'entity',
        'q': q,
        'query_by': 'name, type, description',
    }
    ]
}

# Search parameters that are common to all searches go here
common_search_params =  {
    # 'query_by': 'name',
}

search_hits = client.multi_search.perform(search_requests, common_search_params)
multi_search_result = TypesenseMultiSearchResult.model_validate(search_hits)

In [ ]:
search_hits['results'][1]
# multi_search_result.model_dump()